includes everything except words analysis, 6 hidden nodes, deep learning less acurate, loss= categorical_crossentropy

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("Twitter_Data_Final_v1.xlsx")

In [3]:
df.head()

,label,id,name,username,bio,location,url,join_date,join_time,tweets,...,followers,likes,media,private,verified,profile_image_url,background_image,color_number,face_detection,follower/following_ratio
0,individual,379823034,mohammed,BarcaMohammed,NaN,North Bergen,NaN,2011-09-25,09:33:00,172,...,12,1,1,0,0,https://pbs.twimg.com/profile_images/166601617...,NaN,51912,0.0,0.023256
1,individual,931229576,SaMisches,Shmisch,NaN,NaN,NaN,2012-11-06,20:41:00,6,...,10,3,3,0,0,https://pbs.twimg.com/profile_images/930857677...,NaN,31033,0.0,0.108696
2,individual,393095898,Christopher Pawelski,ChrisPawelski,A 4th generation family onion farmer married t...,"Florida, NY",http://about.me/chris_pawelski,2011-10-17,18:00:00,49503,...,63997,30888,23400,0,0,https://pbs.twimg.com/profile_images/686658233...,https://pbs.twimg.com/profile_banners/39309589...,46205,0.0,1.085431
3,individual,3109060051,xrishin,xrishin,NaN,NaN,NaN,2015-03-26,19:39:00,3,...,2,0,0,1,0,https://abs.twimg.com/sticky/default_profile_i...,NaN,184,0.0,0.050000
4,business,1133740000000000000,LENDVER,LendverLLC,BORROW CONFIDENTLY. We√¢‚Ç¨‚Ñ¢ve done the rese...,NaN,https://www.lendver.com/,2019-05-29,07:32:00,2,...,2,0,1,0,0,https://pbs.twimg.com/profile_images/113375859...,https://pbs.twimg.com/profile_banners/11337428...,510,0.0,0.090909


In [4]:
df.shape

(11401, 21)

In [5]:
# Check label column for unique values. Previously, "Business" and "business" existed, 
# as did "Indivdual" and "individual". This led to 4 categories during encoding.
df["label"].unique()

array(['individual', 'business', nan], dtype=object)

In [6]:
df.columns

Index(['label', 'id', 'name', 'username', 'bio', 'location', 'url',
       'join_date', 'join_time', 'tweets', 'following', 'followers', 'likes',
       'media', 'private', 'verified', 'profile_image_url', 'background_image',
       'color_number', 'face_detection', 'follower/following_ratio'],
      dtype='object')

In [7]:
# 999 was a code we used when executing python face_detection library on twitter profile pictures. 
# Specifically, 999 indicated that there was an error/no picture found when iterating through thr profile pictures
df = df[df.face_detection != 999]
# code 1 was assigned if a face was deteced, code 0 was assigned if a face was not detected.

In [8]:
# Create a column in dataframe indicating whether a url was linked on the twitter page or not. 
# If a url was detected, code 1 was assigned, else, 0 was assigned.
array = []
for i in list(df["url"]):
    if str(i) != "nan":
        array.append(1)
    else:
        array.append(0)
df["url_detect"] = array

In [9]:
df.shape

(11209, 22)

In [10]:
# color_number column is a column that gives a count of the number of colors a profile picture has.
# We noticed that pictures with 184 colors all linked to the same default twitter profile picture.
# Since no face recognition or insightful color count can be returned and properly evaluated by the model,
# we decided to drop all rows that have a color count of 184.
# Coincidentally, we noticed color_number = 0 all corresponding face_detection = 999.
# Even though that was already dropped, we decided to add this precaution in case this was used on future scraped data
df = df[(df.color_number != 184) & (df.color_number !=0)]

In [11]:
df.shape

(10411, 22)

In [12]:
# Drop columns we dont need for modeling
df = df.drop(columns = ["name", "id", "url", "bio", "location", "join_date", "join_time", \
                        "private", "verified", "profile_image_url", "background_image"])

In [13]:
df = df.dropna(how="any")
# Dataset used had 11401 rows, unclassified as business or not. Our team went into each twitter page to classify
# whether the page was a business or individual. Since 1000s of rows could not be manually classified, 
# we dropped many unclassified rows
# Dropping data must be the last step, because there are blank fields in the dropped columns, which may have values
# in the variables we will take into consideration. Dropping pre-emptively will drop many rows unnecessarily

In [14]:
df.shape

(2359, 11)

In [15]:
df.head()

,label,username,tweets,following,followers,likes,media,color_number,face_detection,follower/following_ratio,url_detect
0,individual,BarcaMohammed,172,516,12,1,1,51912,0.0,0.023256,0
1,individual,Shmisch,6,92,10,3,3,31033,0.0,0.108696,0
2,individual,ChrisPawelski,49503,58960,63997,30888,23400,46205,0.0,1.085431,1
4,business,LendverLLC,2,22,2,0,1,510,0.0,0.090909,1
6,individual,ClaretSport,39675,7607,6964,1750,36600,41897,1.0,0.915473,1


In [16]:
# Assign X (data) and y (target)
X = df.drop(["label", "username"], axis=1)
y = df["label"]
print(X.shape, y.shape)

(2359, 9) (2359,)


In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [18]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)

/Users/kyleeng/anaconda3/envs/PythonData/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


In [19]:
# Transform the training and testing data using the X_scaler

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

/Users/kyleeng/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/kyleeng/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


In [20]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Step 1: Label-encode data set (y_train was previously "business" or "individual", which needed to be numbers) 
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)


# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)


Using TensorFlow backend.


In [21]:
# first, create a normal neural network with 2 inputs, 6 hidden nodes, and 2 outputs
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=6, activation='tanh', input_dim=9))
model.add(Dense(units=2, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 60        
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 14        
Total params: 74
Trainable params: 74
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
y_train_categorical.shape

(1769, 2)

In [25]:
# Fit the model to the training data
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)


Instructions for updating:
Use tf.cast instead.
Epoch 1/75
 - 0s - loss: 0.6037 - acc: 0.7049
Epoch 2/75
 - 0s - loss: 0.4967 - acc: 0.7835
Epoch 3/75
 - 0s - loss: 0.4329 - acc: 0.8219
Epoch 4/75
 - 0s - loss: 0.3901 - acc: 0.8423
Epoch 5/75
 - 0s - loss: 0.3599 - acc: 0.8491
Epoch 6/75
 - 0s - loss: 0.3382 - acc: 0.8575
Epoch 7/75
 - 0s - loss: 0.3226 - acc: 0.8807
Epoch 8/75
 - 0s - loss: 0.3114 - acc: 0.8881
Epoch 9/75
 - 0s - loss: 0.3029 - acc: 0.8869
Epoch 10/75
 - 0s - loss: 0.2966 - acc: 0.8898
Epoch 11/75
 - 0s - loss: 0.2919 - acc: 0.8926
Epoch 12/75
 - 0s - loss: 0.2885 - acc: 0.8915
Epoch 13/75
 - 0s - loss: 0.2855 - acc: 0.8949
Epoch 14/75
 - 0s - loss: 0.2834 - acc: 0.8932
Epoch 15/75
 - 0s - loss: 0.2820 - acc: 0.8920
Epoch 16/75
 - 0s - loss: 0.2806 - acc: 0.8932
Epoch 17/75
 - 0s - loss: 0.2798 - acc: 0.8909
Epoch 18/75
 - 0s - loss: 0.2791 - acc: 0.8909
Epoch 19/75
 - 0s - loss: 0.2784 - acc: 0.8909
Epoch 20/75
 - 0s - loss: 0.2779 - acc: 0.8909
Epoch 21/75
 - 0s - l

In [26]:
deep_model = Sequential()
deep_model.add(Dense(units=2, activation='tanh', input_dim=9))
deep_model.add(Dense(units=2, activation='relu'))
deep_model.add(Dense(units=2, activation='softmax'))

In [27]:
deep_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

deep_model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=10,
    shuffle=True,
    verbose=2
)

Epoch 1/10
 - 0s - loss: 0.8266 - acc: 0.1374
Epoch 2/10
 - 0s - loss: 0.7494 - acc: 0.2951
Epoch 3/10
 - 0s - loss: 0.7091 - acc: 0.5619
Epoch 4/10
 - 0s - loss: 0.6869 - acc: 0.6105
Epoch 5/10
 - 0s - loss: 0.6724 - acc: 0.6360
Epoch 6/10
 - 0s - loss: 0.6635 - acc: 0.6405
Epoch 7/10
 - 0s - loss: 0.6586 - acc: 0.6405
Epoch 8/10
 - 0s - loss: 0.6557 - acc: 0.6410
Epoch 9/10
 - 0s - loss: 0.6538 - acc: 0.6416
Epoch 10/10
 - 0s - loss: 0.6526 - acc: 0.6433


In [28]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

 - 0s - loss: 0.2993 - acc: 0.8898
Normal Neural Network - Loss: 0.29925310798620774, Accuracy: 0.8898305296897888


In [29]:
model_loss, model_accuracy = deep_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

 - 0s - loss: 0.6535 - acc: 0.6407
Deep Neural Network - Loss: 0.653541510185953, Accuracy: 0.6406779885292053


In [30]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier
classifier.fit(X_train, y_train)

# Prediction results on test data set
predictions = classifier.predict(X_test)
comparison = pd.DataFrame({"prediction": predictions, "actual": y_test})
comparison.head()

/Users/kyleeng/anaconda3/envs/PythonData/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


,prediction,actual
921,individual,individual
655,individual,individual
1297,individual,individual
621,individual,individual
7578,individual,individual


In [31]:
# joined dataset so we can see usernames and predictions/actual side by side
merged_comparsion = df.join(comparison, lsuffix='_caller', rsuffix='_other')
merged_comparsion = merged_comparsion[["username", "prediction", "actual"]]
merged_comparsion = merged_comparsion.dropna(how="any")
merged_comparsion.head()

,username,prediction,actual
2,ChrisPawelski,individual,individual
10,Arvidinvest,individual,individual
17,pattyshort,individual,individual
21,HarshnaParoha,individual,individual
23,jsare34,individual,individual


In [32]:
discrepencies = comparison[comparison["prediction"] != comparison["actual"]]
discrepencies.head()

,prediction,actual
1418,individual,business
163,individual,business
1113,individual,business
1923,individual,business
2019,individual,business


In [33]:
# discrepencies_index = list(discrepencies.index)
# discrepencies_index

In [34]:
# df.loc[discrepencies_index].head()

In [35]:
merged_discrepencies = df.join(discrepencies, lsuffix='_caller', rsuffix='_other')
merged_discrepencies = merged_discrepencies[["username", "prediction", "actual"]]
merged_discrepencies = merged_discrepencies.dropna(how="any")
merged_discrepencies.head()

,username,prediction,actual
28,Snap_Cap,individual,business
50,72Services,individual,business
67,RyanHur06436445,individual,business
87,BridgeBayCap,individual,business
94,MissionPOSSlBLE,individual,business


In [36]:
# Create a random forest classifier
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train_scaled, y_train_categorical)
rf.score(X_train_scaled, y_train_categorical)

1.0

In [37]:
# Get weights that machine defined, multiplied by 100 to get percents
importances = rf.feature_importances_*100
importances

array([ 5.69941048,  5.25206931,  4.8670921 ,  6.21140799,  5.64649022,
       29.18841991, 30.22521111,  5.73129157,  7.17860731])

In [38]:
sorted(zip(importances, X), reverse=True)

[(30.225211108623963, 'face_detection'),
 (29.188419911238146, 'color_number'),
 (7.178607311713744, 'url_detect'),
 (6.211407991747113, 'likes'),
 (5.731291574432518, 'follower/following_ratio'),
 (5.699410478320505, 'tweets'),
 (5.646490215150668, 'media'),
 (5.25206931374147, 'following'),
 (4.867092095031887, 'followers')]

In [39]:
# Ideas for presentation: bar graphs of weights, examples of discrepensices with discussion why this may be,
# possible to create graphs like in logistic regression activity? Will need to research when I'm not tired,
# 


In [40]:
unclassified = pd.read_excel("Twitter_Data_Final_v1.xlsx")

unclassified = unclassified[unclassified.face_detection != 999]

array = []
for i in list(unclassified["url"]):
    if str(i) != "nan":
        array.append(1)
    else:
        array.append(0)
unclassified["url_detect"] = array

unclassified = unclassified.drop(columns = ["name", "id", "url", "bio", "location", "join_date", "join_time", \
                        "private", "verified", "profile_image_url", "background_image"])

unclassified = unclassified[unclassified.label != "business"]
unclassified = unclassified[unclassified.label != "individual"]


# unclassified = unclassified.dropna(how="any")

unclassified_X = unclassified.drop(["label", "username"], axis=1)
unclassified.head()

,label,username,tweets,following,followers,likes,media,color_number,face_detection,follower/following_ratio,url_detect
14,NaN,JonnnnyBrown,0,682,33,10,0,184,0.0,0.048387,0
105,NaN,caitiecouch325,13,90,9,29,1,33377,1.0,0.100000,1
137,NaN,SadaKaloZia,695,3861,511,13,231,38147,1.0,0.132349,1
216,NaN,MusketeersTele,103,255,22,110,4,106,0.0,0.086275,0
231,NaN,repport4,14,46,12,21,1,9045,0.0,0.260870,0


In [41]:
unclassified.shape

(8762, 11)

In [42]:
unclassified_X = unclassified_X.dropna(how="any")
unclassified_X.head()
unclassified_X.shape

(8724, 9)

In [43]:
# Predict the class (purple or yellow) of the new data point
unclassified_predictions = classifier.predict(unclassified_X)
unclassified_predictions


unclassified_pred_df = pd.DataFrame({"unclassified predictions": unclassified_predictions})
# unclassified_df.head()

In [44]:
# 

# POSSIBLE INCORRECT JOIN



# 
unclassified_merge = unclassified.join(unclassified_pred_df, lsuffix='_caller', rsuffix='_other')
unclassified_merge = unclassified_merge[["username", "unclassified predictions"]]
# unclassified_df = unclassified_df.dropna(how="any")
unclassified_merge.loc[unclassified_merge["unclassified predictions"] == "business"]

,username,unclassified predictions
14,JonnnnyBrown,business
1388,FinResister,business
1558,TheTaylorRichey,business
1565,NaterTot33,business
1593,worthynow,business
1611,HutchcoDisplays,business
1743,AiAssist,business
1774,QAmaryclements,business
1795,BWARE83,business
1817,InteleLead,business
